In [1]:
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
import pandas as pd
import calendar
import time 


In [2]:
# Initialize gql client
def init_client_v2():
    endpoint='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'
    transport=RequestsHTTPTransport(url=endpoint)
    client = Client(transport=transport, fetch_schema_from_transport=True)
    return client

In [3]:
client = init_client_v2()
query = gql(
    """
    {
        pairs(first: 1000, orderBy: volumeUSD, orderDirection: desc) {
            id,
            token0 {
                symbol
            },
            token1 {
                symbol
            }
            volumeUSD
        }
    }
    """)

result = client.execute(query)
client.close()
pairs = result['pairs']

In [4]:
pairs_df = pd.DataFrame(pairs)

In [5]:
pairs_df.head()

,id,token0,token1,volumeUSD
0,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,{'symbol': 'USDC'},{'symbol': 'WETH'},28854540366.51359144147582542508646
1,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,{'symbol': 'WETH'},{'symbol': 'USDT'},28067155749.86169594329855197136091
2,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,{'symbol': 'DAI'},{'symbol': 'WETH'},13612841868.60976409890595666411258
3,0x23fe4ee3bd9bfd1152993a7954298bb4d426698f,{'symbol': 'SCAMMY'},{'symbol': 'WETH'},11029715648.72159798743878461135205
4,0xe5ffe183ae47f1a0e4194618d34c5b05b98953a8,{'symbol': 'welp'},{'symbol': 'WETH'},11017636252.92521755536995878068531


In [6]:
pairs_df.token0 = [d.get('symbol') for d in pairs_df.token0]
pairs_df.token1 = [d.get('symbol') for d in pairs_df.token1]
pairs_df.head()

,id,token0,token1,volumeUSD
0,0xb4e16d0168e52d35cacd2c6185b44281ec28c9dc,USDC,WETH,28854540366.51359144147582542508646
1,0x0d4a11d5eeaac28ec3f61d100daf4d40471f1852,WETH,USDT,28067155749.86169594329855197136091
2,0xa478c2975ab1ea89e8196811f51a7b7ade33eb11,DAI,WETH,13612841868.60976409890595666411258
3,0x23fe4ee3bd9bfd1152993a7954298bb4d426698f,SCAMMY,WETH,11029715648.72159798743878461135205
4,0xe5ffe183ae47f1a0e4194618d34c5b05b98953a8,welp,WETH,11017636252.92521755536995878068531


In [8]:
pairs_df[25:35]

,id,token0,token1,volumeUSD
25,0x819f3450da6f110ba6ea52195b3beafa246062de,MATIC,WETH,1083107282.029894526204302223801658
26,0x4d5ef58aac27d99935e5b6b4a6778ff292059991,DPI,WETH,1075757629.439227423904837127959137
27,0x570febdf89c07f256c75686caca215289bb11cfc,ERN,WETH,1021935980.7198211161920127448636
28,0xffa98a091331df4600f87c9164cd27e8a5cd2405,POLS,WETH,1005056035.610445629916869749988211
29,0x43ae24960e5534731fc831386c07755a2dc33d47,SNX,WETH,963866708.0193042322150492422962201
30,0x25647e01bd0967c1b9599fa3521939871d1d0888,WETH,SUPER,938181244.6280214631037733651500069
31,0x87febfb3ac5791034fd5ef1a615e9d9627c2665d,KP3R,WETH,860410300.4494357753361886205675997
32,0x32ce7e48debdccbfe0cd037cc89526e4382cb81b,CORE,WETH,857161737.2665242850323818162522315
33,0x7ba9b94127d434182287de708643932ec036d365,eRSDL,WETH,842249633.5239177384389459454164888
34,0x70ec2fa6eccf4010eaf572d1c1a7bcbc72dec983,WETH,ROOK,826235889.2200002573184410649966573


In [9]:
def get_pair_day_data_v2(pair_address, iteration=1):
    """
        Get 1000 days worth of data
        If len(result) < 1000, you've probably gotten all the data there is
    """
    client = init_client_v2()

    daily_APYs = []
    current_time = calendar.timegm(time.gmtime())

    # Calculate epoch time of date deltas 
    end_date = current_time - (86400000 * iteration) # 1000 days in seconds 



    query = gql(
    """
    query ($address: Bytes!, $enddate: Int!) {
        pairDayDatas(first: 1000, orderBy: date, orderDirection: desc,
        where: {
            pairAddress: $address,
            date_gt: $enddate
    }
    ) {
        date
        token0 {
            symbol
        }
        token1 {
            symbol
        }
        dailyVolumeToken0
        dailyVolumeToken1
        dailyVolumeUSD
        totalSupply
        reserveUSD
    }
    }
    """)
    params = {
        "address": pair_address,
        "enddate": end_date,
        }

    result = client.execute(query, variable_values=params)

    client.close()



    if len(result["pairDayDatas"]) < 1000:

        iteration = 'Done'
        return result['pairDayDatas'], iteration

    else:
        iteration = iteration + 1
        return result['pairDayDatas'], iteration

In [15]:
first1k, iteration = get_pair_day_data_v2(pair_address="0x4d5ef58aac27d99935e5b6b4a6778ff292059991")
iteration

'Done'

In [16]:
first1k = pd.DataFrame(first1k)
first1k.head()

,dailyVolumeToken0,dailyVolumeToken1,dailyVolumeUSD,date,reserveUSD,token0,token1,totalSupply
0,2944.213249743356440057,360.144078298713477596,784600.6789943743383296287166172914,1624924800,37109056.88685369084093119830349348,{'symbol': 'DPI'},{'symbol': 'WETH'},21371.005923015243261765
1,5437.271014231465486119,649.100699516474158638,1339716.213228797761038887255690849,1624838400,34571866.75960446358416072854982725,{'symbol': 'DPI'},{'symbol': 'WETH'},21367.473371469439379609
2,3072.416788572231461384,367.562810941605667462,682989.3406043296910399800708753965,1624752000,32113248.52413519972948351324872476,{'symbol': 'DPI'},{'symbol': 'WETH'},21303.584282037574584897
3,5149.713099684089287711,613.891755432629820163,1094919.756300682326846168043425021,1624665600,29208446.81454486943232636279218528,{'symbol': 'DPI'},{'symbol': 'WETH'},21031.16612878440030901
4,3573.975753676325264567,427.968327182256991692,803500.3008454801623893316355888608,1624579200,28386059.61846493533624861216184274,{'symbol': 'DPI'},{'symbol': 'WETH'},20267.457979770847007021


In [17]:
first1k.token0 = [d.get('symbol') for d in first1k.token0]
first1k.token1 = [d.get('symbol') for d in first1k.token1]
first1k.head()

,dailyVolumeToken0,dailyVolumeToken1,dailyVolumeUSD,date,reserveUSD,token0,token1,totalSupply
0,2944.213249743356440057,360.144078298713477596,784600.6789943743383296287166172914,1624924800,37109056.88685369084093119830349348,DPI,WETH,21371.005923015243261765
1,5437.271014231465486119,649.100699516474158638,1339716.213228797761038887255690849,1624838400,34571866.75960446358416072854982725,DPI,WETH,21367.473371469439379609
2,3072.416788572231461384,367.562810941605667462,682989.3406043296910399800708753965,1624752000,32113248.52413519972948351324872476,DPI,WETH,21303.584282037574584897
3,5149.713099684089287711,613.891755432629820163,1094919.756300682326846168043425021,1624665600,29208446.81454486943232636279218528,DPI,WETH,21031.16612878440030901
4,3573.975753676325264567,427.968327182256991692,803500.3008454801623893316355888608,1624579200,28386059.61846493533624861216184274,DPI,WETH,20267.457979770847007021


In [18]:
first1k.shape

(293, 8)

In [19]:
first1k.to_csv("../data/processed/dpi-weth.csv")